In [1]:
# Working with the Full Data (sex & Age) via Excel & Pickle files
#For this work, starting from A_Wip_D
# Finalized on 09/08/2019 @ 9:10 PM

In [2]:
import pandas as pd
import numpy as np
import string as str
import re

import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import widgets, interactive, Layout

In [3]:
pickle_file = 'Full_Data.pkl'

In [4]:
Full_Data_df = pd.read_pickle(pickle_file)
Full_Data_df.rename(columns={'Unknown': 'Unknown_Sex'}, inplace=True)

In [5]:
cols = list(Full_Data_df.columns.values)

In [6]:
#Swap two columnss Offense and Unknown_Sex
hdg1="Offense"
hdg2="Unknown_Sex"
indx1=cols.index(hdg1)
indx2=cols.index(hdg2)
print(indx1, indx2)
cols[indx1]=hdg2
cols[indx2]=hdg1

3 4


In [7]:
Full_Data_df=Full_Data_df.reindex(columns=cols)

In [8]:
offense_types=list(Full_Data_df["Offense"].unique())

In [9]:
min_year=1991
max_year=2017
quitcontrol="QUIT---All Done"
offtype = widgets.Dropdown(
    options=[quitcontrol]+offense_types+[quitcontrol],
    value='all-offenses',
    description='Offense:',
)

start_year = widgets.BoundedFloatText(
    value=min_year,                        
    min=min_year,                          
    max=max_year-1,                          
    step=1,
    description='Start Year:',
    disabled=False,
    color='black'
)
end_year = widgets.BoundedFloatText(
    value=max_year,                        
    min=min_year+1,                          
    max=max_year,                          
    step=1,
    description='End Year:',
    disabled=False,
    color='black'
)

In [10]:
def plotit(offtype, start_year, end_year):
    df=Full_Data_df[Full_Data_df["Offense"]==offtype].copy()
    
    if end_year <= start_year:
        if start_year < max_year: 
            end_year=start_year+1
        else:
            start_year=end_year-1
        
    df=df[df["Year"]>=start_year]
    df=df[df["Year"]<=end_year]     
    df.to_csv("offenders_by_age_"+offtype+".CSV")
    
    dfgrpbytype=df.groupby('Year').sum()
    dfbytype=dfgrpbytype.copy()
   
    indx=cols.index("Offense")
    dfsex=dfbytype.iloc[:,:indx]
    dfage=dfbytype.iloc[:,indx:]
   
    xvals=list(dfage.index)
    
    syr="{:.0f}".format(start_year)
    eyr="{:.0f}".format(end_year)

    fig1, ax1 = plt.subplots(figsize=(10,5.8))
    
    plt.xlim((1990, 2018))
    plt.xlabel("Years")
    plt.ylabel("# of Cases")
    plt.title("FBI Cases ("+syr+" - "+eyr+") for: "+offtype.title())
    plt.grid(b=True, which='major', color='#666666', linestyle=':')
    
    lgx=1.2
    lgy=1.02
    
    dfage=dfbytype.copy()
    xn=0
    sextotal=0
    agetotal=0
    for col in dfage.columns: 
        yvals=dfage[col]
        thistotal=yvals.sum()
        totalx="{:,}".format(thistotal)
        ctotal=totalx.rjust(16,".")
        
        lgtitl=col.replace("range","age").title()
        lgtitle=lgtitl.ljust(16,".")+ctotal
        if xn==2: lgtitle=lgtitle+"\n"
        if xn < 3: 
            ls="dashed" 
            dfage[col].plot(x=xvals,ax=ax1, label=lgtitle,linestyle=ls)
            sextotal=sextotal+thistotal
        else: 
            if xn < 8:
                ls="solid"
                ax1.plot(xvals, yvals, label=lgtitle,linestyle=ls)
                agetotal=agetotal+thistotal
            else:
                ls="dashdot"  #"dotted"
                ax1.plot(xvals, yvals, label=lgtitle,linestyle=ls)
                agetotal=agetotal+thistotal
        xn=xn+1
    
    sexgt=("By Sex Grand Total: "+"{:,}".format(sextotal))
    agegt=("By Age Grand Total: "+"{:,}".format(agetotal))
    lghdg="Numbers shown represents totals for\neach category summed over all years\n\n"+sexgt+"\n"+agegt+"\n\n"
    plt.legend(loc='upper center', bbox_to_anchor=(lgx, lgy),ncol=1, fancybox=True, shadow=True, title=lghdg)
    ax1.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
    ax1.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:.0f}".format(int(x))))
    plt.savefig('FBI_Data_1991-2017_'+offtype+'.png', bbox_inches='tight')
    plt.show()
    
#     controls = HBox(widget.children[:-1], layout = Layout(flex_flow='row wrap'))
#     output = widget.children[-1]
#     display(VBox([controls, output]))

In [11]:
def pre_plotit(offtype, start_year, end_year):
    if offtype==quitcontrol:
        print("\n\n\n All Done & Quitting .............. \n\n Good Bye! \n\n")
    else:
        plotit(offtype, start_year, end_year)


In [12]:
interactive(pre_plotit, offtype=offtype, start_year=start_year, end_year=end_year)

interactive(children=(Dropdown(description='Offense:', index=64, options=('QUIT---All Done', 'aggravated-assau…